In [2]:
import numpy as np
import theano.tensor as T
import theano

In [6]:
x = T.dmatrix("x")
s = 1. / (1. + T.exp(-x))
logistic = theano.function([x], s)
print(logistic([[0., 1.], [-1., -2.]]))
s2 = (1 + T.tanh(x/2)) / 2
logistic2 = theano.function([x], s2)
print(logistic2([[0., 1.], [-1., -2.]]))

[[ 0.5         0.73105858]
 [ 0.26894142  0.11920292]]
[[ 0.5         0.73105858]
 [ 0.26894142  0.11920292]]


In [9]:
a, b = T.dmatrices('a', 'b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
f = theano.function([a, b], [diff, abs_diff, diff_squared])
f([[5, 2]], [[2, 5]])

[array([[ 3., -3.]]), array([[ 3.,  3.]]), array([[ 9.,  9.]])]

In [15]:
x, y = T.dscalars('x', 'y')
z = x + y
f = theano.function([x, theano.In(y, value=1)], z)
f(10), f(10,1)

(array(11.0), array(11.0))

In [24]:
x, y, w = T.scalars('x', 'y_not', 'w')
z = (x + y) * w
f = theano.function([x, theano.In(y, value=1), theano.In(w, value=2, name="w_by_name")], z)
f(10), f(10, w_by_name=3), f(10, w_by_name=3, y_not=2)

(array(22.0), array(33.0), array(36.0))

In [50]:
state = theano.shared(0)
inc = T.iscalar('inc')
accumulator = theano.function([inc], state, updates=[(state, state+inc)], on_unused_input='warn') # updates takes '(shared-var, new-expression)'


In [31]:
state.set_value(0)
print(state.get_value())
print(accumulator(1)) # prints current state value not new value
print(state.get_value())
print(accumulator(300))
print(state.get_value())

0
0
1
1
301


In [33]:
decrementor = theano.function([inc], state, updates=[(state, state-inc)])
state.set_value(0)
accumulator(10)
print(state.get_value())
decrementor(5)
print(state.get_value())
decrementor(-10)
print(state.get_value())
accumulator(-5)
print(state.get_value())

10
5
15
10


In [37]:
fn_of_state = state * 2 + inc
foo = T.scalar(dtype = state.dtype)
skip_shared = theano.function([inc, foo], fn_of_state, givens=[(state, foo)])
skip_shared(1, 3), state.get_value()

(array(7), array(10))

In [43]:
state.set_value(0), state.get_value()
accumulator(10), print(state.get_value())
new_state = theano.shared(0)
new_accumulator = accumulator.copy(swap={state:new_state})
new_accumulator(100), print(new_state.get_value(), state.get_value())


10
100 10


([array(0)], None)

In [53]:
null_accumulator = accumulator.copy(delete_updates=True)
print(null_accumulator(9000), state.get_value())

[array(0)] 0


/home/david/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: UserWarning: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 0 is not part of the computational graph needed to compute the outputs: inc.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.
  if self.run_code(code, result):


In [60]:
from theano.tensor.shared_randomstreams import RandomStreams
from theano import function
srng = RandomStreams(seed=12543)
rv_u = srng.uniform((2, 2))
rv_n = srng.normal((2, 2))
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)
nearly_zeros = function([], rv_u + rv_u - 2 * rv_u)
f_val0 = f()
f_val1 = f()
g_val0 = g()
g_val1 = g()
print(f_val0, f_val1)
print(g_val0, g_val1)
print(nearly_zeros())

[[ 0.60021117  0.97288269]
 [ 0.52115434  0.30299242]] [[ 0.19618627  0.71826149]
 [ 0.08793276  0.73265941]]
[[ 1.08775828 -0.90589691]
 [ 0.69614725 -0.48121802]] [[ 1.08775828 -0.90589691]
 [ 0.69614725 -0.48121802]]
[[ 0.  0.]
 [ 0.  0.]]


In [62]:
state_after_v0 = rv_u.rng.get_value().get_state()
print(nearly_zeros())
v1 = f()
rng = rv_u.rng.get_value(borrow=True)
rng.set_state(state_after_v0)
rv_u.rng.set_value(rng, borrow=True)
v2 = f()
v3 = f()
print(v1, v2, v3)


[[ 0.  0.]
 [ 0.  0.]]
[[ 0.18770837  0.33941054]
 [ 0.03833877  0.12951773]] [[ 0.65937998  0.05769814]
 [ 0.62882717  0.55863177]] [[ 0.18770837  0.33941054]
 [ 0.03833877  0.12951773]]
